# TP 3 de Statistique descriptive avec SAS

Objectifs de la séance : 
* blabbla
* blablabla

TO DO : bien faire une présentation de la base et des variables des tables equipement, libelle et population

## Analyse univariée du nombre d'équipements

**Question 1** : Créez une bibliothèque *source* qui contiendra les données en lecture seule. Créez également une bibliothèque *resultat* qui contiendra les résultats de vos exercices

In [ ]:
LIBNAME source "/folders/myfolders/Donnees_TP_SAS/TP3/source" ACCESS = READONLY;
LIBNAME resultat "/folders/myfolders/Donnees_TP_SAS/TP3/resultat";

**Question 2** : Affichez les dix premières lignes de la table *equipement* et affichez des informations générales dessus. 

In [ ]:
PROC PRINT DATA = source.equipement(obs = 10);
RUN;

In [ ]:
PROC CONTENTS DATA = source.equipement;
RUN;

**Question 3** : La variable COMINSEE correspond au code géographique de la commune (il est différent du code postal). Néanmoins, ce code est censé être sur cinq positions (i.e de la forme xxxxx). Est-ce le cas pour toutes les communes ? Utilisez un format prédéfini adapté pour afficher correctement les codes géographiques. 

La table après application du format se nommera *equipement* et sera stockée dans la bibliothèque *resultat*.

**Réponse** : L'idée est de faire utiliser le format *Zn.* Je l'avais évoqué en cours de SAS début septembre.

In [ ]:
DATA resultat.equipement;
SET source.equipement;
FORMAT COMINSEE Z5.;
RUN;

**Question** : La table *libelle* contient deux variables permettant de faire le lien entre le numéro d'une activité (*Numero*) et le nom de l'activité (*Libelle*). À partir de cette table, créez le format *nomactivite* qui à un numéro d'activité associera le nom de l'activité.

In [ ]:
DATA format_lien;
SET source.libelle;
fmtname = "nomactivite";
RENAME numero = start
libelle = label;
RUN;

PROC FORMAT CNTLIN = format_lien;
RUN;

**Question** : Calculez le nombre total d'équipements par commune et stockez les résultats dans une table *nb_eq_com* dans la bibliothèque *resultat*. On n'affichera pas les résultats.
La table *nb_eq_com* ne contiendra que deux variables : ComInsee (numéro de la commune) et nb_equip_tot (nombre total d'équipements).

In [ ]:
PROC MEANS DATA = source.equipement SUM NOPRINT;
VAR EFFECTIF;
BY COMINSEE;
OUTPUT OUT = resultat.nb_eq_com(keep=COMINSEE nb_equip_tot) SUM = nb_equip_tot;
RUN;

**Question** : À l'aide d'une étape DATA, ajoutez le libellé "Code géographique officiel (COG)" à la variable *COMINSEE* et le libellé "Nombre d'équipements" à la variable *nb_equip_tot*.

In [ ]:
DATA resultat.nb_eq_com;
SET resultat.nb_eq_com;
LABEL COMINSEE = "Code géographique officiel (COG)"
nb_equip_tot = "Nombre d'équipements";
RUN;

**Question** : Calculez le nombre d'équipements par établissement. On stockera les résultats dans la table *nb_eq_dep* dans la bibliothèque *resultat*.

Remarque : Le département correspond aux deux premières positions du code géographique officiel (donc sur cinq positions). On pourra ajouter une variable *dep* à la table *nb_eq_com* en utilisant les fonctions PUT et SUBSTR.

**Note pour les chargés de TP** : Cette question est intéressante dans la mesure où elle rappelle que si les élèves utilisent la fonction SUBSTR directement sur la variable COMINSEE alors on n'obtiendra pas le résultat voulu car la variable est numérique. Il faut donc la transformer en caractère en utilisant la fonction PUT et le format Z5. (pour bien afficher le cog sur 5 positions) puis appliquer la fonction SUBSTR en ne gardant que les deux premières positions.

In [ ]:
DATA resultat.nb_eq_com;
SET resultat.nb_eq_com;
cominsee_carac = PUT(COMINSEE, Z5.);
DEP = SUBSTR(cominsee_carac, 1,2);
DROP cominsee_carac;
RUN;

In [ ]:
proc contents data = resultat.nb_eq_com;
run;

In [ ]:
PROC MEANS DATA = resultat.nb_eq_com NOPRINT;
VAR nb_equip_tot;
BY DEP;
OUTPUT OUT = resultat.nb_eq_dep(keep=DEP nb_equip_tot_dep) SUM = nb_equip_tot_dep;
RUN;

In [ ]:
proc print data = resultat.nb_eq_dep;
run;

## Analyse de la population par département

**Question** : Affichez des informations générales sur la table *population*.

In [ ]:
PROC CONTENTS data = source.population;
RUN;

**Question** : Ajoutez une variable de type texte COMINSEE qui contiendra le code officiel géographique de chaque commune. Cette nouvelle table sera enregistré dans la bibliothèque *resultat* et se nommera *population*. 

Le code officiel géographique est la concaténation du code du département et du code de la commune.

In [ ]:
DATA resultat.population;
SET source.population;
LENGTH COMINSEE $5;
COMINSEE = CATS(code_dep,code_com);
RUN;

In [ ]:
PROC PRINT DATA = resultat.population(obs = 20);
RUN;

**Question** : Créez la table *pop_equip_com* dans la bibliothèque *resultat* qui contiendra pour chaque observation (ici les communes) : 
* une variable ComInsee qui correspondra au code géographique officiel de la commune,
* une variable nb_equip_tot qui correspondra au nombre total d'équipements de la commune,
* une variable pop_tot qui correspondra au nombre d'habitants de la commune.

On ne gardera que les communes dont on connaît le nombre d'équipements. On se servira de la table *population* créée à la question précédente.

**Note pour les chargés de TP** : L'idée est qu'ils fassent un MERGE entre la table population et la table nb_eq_com. Le problème est que dans la table nb_eq_com la variable COMINSEE est numérique alors qu'elle est caractère dans la table population. Ils devront utiliser les fonctions PUT et INPUT avant !

Bien faire attention à définir la taille des variables.

Dans le cours j'ai parlé de l'opérateur !! pour concaténer deux chaînes de caractères. Là, il faut faire une concaténation sans espace donc utiliser la fonction CATS ... Laissez les faire une petite recherche sur Internet.

In [ ]:
DATA nb_eq_com_tempo;
SET resultat.nb_eq_com;
LENGTH COMINSEE2 $5;
COMINSEE2 = PUT(COMINSEE, Z5.);
DROP COMINSEE;
RENAME COMINSEE2 = COMINSEE;
RUN;

In [ ]:
PROC SORT DATA = nb_eq_com_tempo;
BY COMINSEE;
RUN;

PROC SORT DATA = resultat.population;
BY COMINSEE;
RUN;

DATA resultat.pop_equi_com;
MERGE nb_eq_com_tempo(in = eq) resultat.population(in = pop);
IF eq then OUTPUT;
KEEP COMINSEE NB_EQUIP_TOT POP_TOT;
BY COMINSEE;
RUN;

**Question** : De quels types (quantitatives ou qualitatives) sont les variables NB_EQUIP_TOT et POP_TOT ? 

**Question** : À l'aide d'un graphique, mettez en lumière le lien entre ces deux variables.

**Note pour les chargés de TP** : 